In [1]:
import re
import PyPDF2
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk import sent_tokenize, word_tokenize, pos_tag

In [2]:
def pdf_to_text(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    text = text.replace('\n', ' ')
    return text

In [3]:
text = pdf_to_text("'Kozmopolitanizmin ____kin Gerilimi____ Politikan__n __a__da__ Krizini Kavram Tarihi I________nda D______nmek[#927454]-1731902.pdf")

In [4]:
text

'33 Başkır Ü D (2021).  ‘Kozmopolitanizmin İçkin Gerilimi’: Politikanın Çağdaş Krizini  Kavram Tarihi Işığında Düşünmek. Mülkiye Dergisi , 45 (1), 33-60.‘Kozmopolitanizmin İçkin Gerilimi’: Politikanın  Çağdaş Krizini Kavram Tarihi Işığında Düşünmek1 Ünsal Doğan Başkır , İzmir Ekonomi Üniversitesi, Rektörlük,   ORCID 0000-0003-0779-1450, dogan.baskir@ieu.edu.tr Özet Çağımız, iklim krizinden radikal sağ popülizmin patolojilerine uzanan bir hatta  politikanın tüm dünyaya yayılmış ve yerküre ölçeğinde deneyimlenen krizlerine sahne  olmakta. Bu krizler politikanın çözüm üretmek konusunda tıkanmasına yol açtığı gibi,  yeni politik alternatifleri ve farklı politika tahayyüllerini tartışmak için de olanaklar  sunuyor. Bu çalışma, çağımızın politik krizleri karşısında kozmopolitan tahayyülün  önerebileceklerini kavram tarihsel bir perspektifle ele almayı amaçlıyor. Kabaca ahlaki,  kültürel ve politik ölçeklerde yerel aidiyetleri sorgulayan ve aşan bir bakış açısı ortaya  koyan kozmopolitanizm, 

In [5]:
def find_article_title(article_text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article_text)
    if sentences:
        return sentences[0].strip()
    else:
        return "Title not found"

In [6]:
title = find_article_title(text)

In [7]:
title

'33 Başkır Ü D (2021).'

In [8]:
def find_author_name(text):
    sentences = sent_tokenize(text)
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged = pos_tag(words)
        named_entities = nltk.ne_chunk(tagged)
        for entity in named_entities:
            if isinstance(entity, nltk.Tree) and entity.label() == "PERSON":
                author_name = " ".join([leaf[0] for leaf in entity.leaves()])
                return author_name
    return ""

In [9]:
author_name = find_author_name(text)

In [10]:
author_name

'Başkır Ü D'

In [11]:
def split_article(article_text):
    reference_pattern = r"Kaynakça|References|Bibliography|Citations|Sonnotlar"

    reference_match = re.search(reference_pattern, article_text, re.IGNORECASE)
    
    if reference_match:
        main_text = article_text[:reference_match.start()]
        reference_section = article_text[reference_match.start():]
        
        return main_text, reference_section
    else:
        return article_text, None

In [12]:
text, references = split_article(text)

In [13]:
def find_references(text,title,author_name):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    sentences_with_references = [sentence for sentence in sentences if pattern.search(sentence) and
                                 sentence != title
                                 and author_name not in sentence #exclude author?
                                 and cosine_similarity(CountVectorizer().fit_transform([title, sentence]))[0][1]<0.6
                                 #???trying to exclude title based on the similarity???
                                ]
    return sentences_with_references

sentences_with_references = find_references(text,title,author_name)

In [14]:
for i in sentences_with_references:
    print(i)
    print("--------")

Ulusal  politikalarımıza kapanmamızla aşamayacağımız derinlikte bir krizle karşı  karşıya olduğumuz yönündeki bu tespit bizi ikinci bir noktaya taşıyor: Reinhardt  Kosselleck’in (2006: 357-358) hatırlattığı üzere, kriz kavramı (tıpkı “eleştiri”  [Kritik ] kavramı gibi) eski Yunanca “ayırmak”, “seçmek”, “yargıda bulunmak”  gibi anlamlara gelen κρίνω/krinó  kökünden türemiştir.
--------
Öte yandan hangi tarihsel ve  düşünsel bağlamda tartışılırsa tartışılsın, kozmopolitanizmin ait olunan topluluk  karşısında mesafe almayı (Anderson, 1998: 267) ve eleştirel bir zeminde insan  olmak bakımından ortaya çıkan haklar ve sorumluluklar konusunda düşünmeyi  önerdiğine şüphe yok.
--------
Buna göre, “kavram olmadan tecrübe olmaz ve  tecrübe olmadan kavram olmaz” (Koselleck, 2009: 58).
--------
Kavram tarihi ekolünün asıl sorusu da burada  belirir: “Kavramlar ile olgusal içerikler arasındaki zamansal ilişki nasıl biçimlenir”  (Koselleck, 2009: 60)?
--------
Mülkiye Dergisi , 45 (1), 33-60.dünya yur